In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from google.colab import files
uploaded = files.upload()
print(uploaded)

In [ ]:
df = pd.read_csv('all_stocks_5yr.csv')

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)


In [ ]:
df.head()

In [ ]:
df['return'] = df['close'].pct_change()


In [ ]:
df.head()

In [ ]:
window = 10
df['Momentum_Signal'] = np.where(df['close'].pct_change(window) > 0, 1, 0)


In [ ]:
df['Momentum_Strategy'] = df['Momentum_Signal'].shift(1) * df['return']


In [ ]:
df['Momentum_Cum'] = (1 + df['Momentum_Strategy']).cumprod()
df['SP500_Cum'] = (1 + df['return']).cumprod()


In [ ]:
ma_window = 20
df['MA20'] = df['close'].rolling(ma_window).mean()


In [ ]:
df['MeanRev_Signal'] = np.where(df['close'] < df['MA20'], 1, -1)  # buy if below MA, sell if above


In [ ]:
df['MeanRev_Strategy'] = df['MeanRev_Signal'].shift(1) * df['return']
df['MeanRev_Cum'] = (1 + df['MeanRev_Strategy']).cumprod()


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['SP500_Cum'], label='S&P 500')
plt.plot(df['Momentum_Cum'], label='Momentum Strategy')
plt.plot(df['MeanRev_Cum'], label='Mean-Reversion Strategy')
plt.legend()
plt.show()
